In [2]:
#파이썬 버전 확인
!python --version

Python 3.6.13 :: Anaconda, Inc.


In [3]:
!pip install --upgrade pip

In [4]:
#- Kkma import 작업해주기
!python --version

!pip install --upgrade pip

!pip install JPype1-1.1.2-cp38-cp38-win_amd64.whl

!pip install konlpy

from konlpy.tag import Kkma

kkma = Kkma()

Python 3.6.13 :: Anaconda, Inc.


ERROR: JPype1-1.1.2-cp38-cp38-win_amd64.whl is not a supported wheel on this platform.


SystemError: java.lang.UnsatisfiedLinkError: Native Library C:\Users\SM2127\anaconda3\envs\deep01\Lib\site-packages\_jpype.cp36-win_amd64.pyd already loaded in another classloader

In [4]:
import pandas as pd

In [8]:
df_train = pd.read_csv('nsmc-master/ratings_train.txt', delimiter='\t')

In [11]:
df_train.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [15]:
#결측치 제거 
# .dropna : 결측값있는 행 제거
df_train.dropna(inplace=True)

In [17]:
df_test = pd.read_csv('nsmc-master/ratings_test.txt', delimiter='\t')

In [19]:
df_test.dropna(inplace=True)

In [20]:
X_train = df_train['document']
y_train = df_train['label']
X_test = df_test['document']
y_test = df_test['label']
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(149995,)
(149995,)
(49997,)
(49997,)


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
tfidf = TfidfVectorizer()
tfidf.fit(X_train[:3])

TfidfVectorizer()

In [27]:
X_train[:3]

0                  아 더빙.. 진짜 짜증나네요 목소리
1    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                    너무재밓었다그래서보는것을추천한다
Name: document, dtype: object

In [26]:
#4-1) 띄어쓰기 기준으로 토큰화 하여 단어사진 확인
tfidf.vocabulary_

{'더빙': 2,
 '진짜': 6,
 '짜증나네요': 7,
 '목소리': 3,
 '포스터보고': 9,
 '초딩영화줄': 8,
 '오버연기조차': 5,
 '가볍지': 0,
 '않구나': 4,
 '너무재밓었다그래서보는것을추천한다': 1}

In [28]:
kkma = Kkma()

In [29]:
#:: kkma ::
- 토큰화를 조금 더 유기적으로 해줌(한글에 맞게)
- kkma 토큰화 방법 총 3가지 있음
# 문장단위 토큰화: sentences()
# 명사단위 토큰화: nouns()
# 형태소단위 토큰화: morphs()


In [31]:
X_train[0]

'아 더빙.. 진짜 짜증나네요 목소리'

In [32]:
kkma.nouns(X_train[0])

['더빙', '목소리']

In [30]:
kkma.morphs(X_train[0])

['아', '아', '더빙', '..', '진짜', '짜증나', '네요', '목소리']

In [33]:
#- .pos : 나뉜 형태소가 어떤걸 의미하는지 적혀있음
kkma.pos(X_train[0])
# - 감성분석할 때는 동사나 형용사만 가져오면됨

[('아', 'VV'),
 ('아', 'ECS'),
 ('더빙', 'NNG'),
 ('..', 'SW'),
 ('진짜', 'MAG'),
 ('짜증나', 'VV'),
 ('네요', 'EFN'),
 ('목소리', 'NNG')]

In [39]:
d=pd.DataFrame(kkma.pos(X_train[0]),columns=['morphs','tag'])

In [35]:
d.set_index('tag',inplace=True)

In [37]:
d.loc[['VV','NNG']]

,morphs
tag,
VV,아
VV,짜증나
NNG,더빙
NNG,목소리


In [40]:
# tfidf의 토큰화 방법 변경

In [41]:
# 명사 단위로 토큰화
def myTokenizer(text):
    return kkma.nouns(text)

In [42]:
# tfidf 내가 만든 토큰방법 적용
tfidf = TfidfVectorizer(tokenizer = myTokenizer)

In [43]:
tfidf.fit(X_train[:3])
tfidf.vocabulary_ 

C:\Users\SM2127\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


{'더빙': 0,
 '목소리': 2,
 '흠': 17,
 '포스터': 15,
 '포스터보고': 16,
 '보고': 4,
 '초': 12,
 '초딩영화줄': 13,
 '딩': 1,
 '영화': 6,
 '줄': 11,
 '오버': 7,
 '오버연기': 8,
 '연기': 5,
 '재': 9,
 '재밓': 10,
 '밓': 3,
 '추천': 14}

In [44]:
# VV,VA,NNG의 형태소만 토큰화
def myTokenizer2(text):
    d = pd.DataFrame(kkma.pos(text), columns=['morph','tag'])
    d.set_index('tag', inplace=True)
    if ('VV' in d.index) | ('VA' in d.index) | ('NNG' in d.index):
        labels = ['VV','VA','NNG']
        return d.loc[d.index.intersection(labels)].dropna()['morph'].values
    #intersection교집합연산
    else :
        return []

In [46]:
# 3개의 문장 학습시켜서 단어사전 형태로 출력하기
tfidf = TfidfVectorizer(tokenizer = myTokenizer2)
tfidf.fit(X_train[:3])
tfidf.vocabulary_

C:\Users\SM2127\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


{'아': 5,
 '짜증나': 12,
 '더빙': 1,
 '목소리': 2,
 '흠': 15,
 '포스터': 14,
 '보고': 4,
 '영화': 8,
 '줄': 11,
 '오버': 9,
 '연기': 7,
 '가볍': 0,
 '재': 10,
 '추천': 13,
 '어': 6,
 '보': 3}

In [47]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

In [49]:
pipe = make_pipeline(TfidfVectorizer(tokenizer = myTokenizer2),LinearSVC())

In [50]:
pipe.fit(X_train[:100],y_train[:100])
# .steps : 결과물을 확인하기 위해 몇번째 단계에 어떤걸 사용했는지 확인하고 싶을 때 확인하는 함수

#- pipeline에 연결된 tfidfvectorizer, linearsvc 각자 따로 확인할 수 있음

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(tokenizer=<function myTokenizer2 at 0x00000221E8B3E5E0>)),
                ('linearsvc', LinearSVC())])

In [65]:
# steps몇번째 단계에 검색하고 싶을때 확인하는 함수
#--> pipeline에 연결되어있는 tfidf는 0행 1열에 있음

#--> pipeline에 연결되어있는 svm는 1행 1열에 있음

[('tfidfvectorizer',
  TfidfVectorizer(tokenizer=<function myTokenizer2 at 0x00000221E8B3E5E0>)),
 ('linearsvc', LinearSVC())]

In [56]:
# tfidf
pipe_tfidf = pipe.steps[0][1]#tfidf를 학습시킨 결과물을 보고싶을때
# svm
pipe_svm = pipe.steps[1][1]#svm을 학습시킨 결과물을 보고싶을때

In [59]:
voc=pd.DataFrame(pipe_tfidf.vocabulary_.keys(),
            index = pipe_tfidf.vocabulary_.values())

In [60]:
# Series로 확인
voc.sort_index()[0]

0         ㄱ
1         ㅇ
2         ㅈ
3         가
4        가볍
       ... 
495      활력
496    횡단보도
497       흠
498      흥행
499       희
Name: 0, Length: 500, dtype: object

In [61]:
# 토큰들과 가중치를 연결해서 
#각각의 토큰들이 긍정 / 부정에 얼마나 영향을 끼쳤는지 확인


In [63]:
result = pd.DataFrame(pipe_svm.coef_.T, index = voc.sort_index()[0],
            columns = ['w'])#.T행과 열을 뒤집어 줌

In [64]:
result.sort_values(by='w',ascending = False)

,w
0,
재미있,0.928460
재밌,0.895684
인상적,0.761269
드라마,0.725359
내일,0.661585
...,...
추천,-0.673391
실망,-0.675712
졸작,-0.675714
